## Environment Setup

In [1]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount("/content/drive")

pd.set_option('mode.chained_assignment', None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
subsDf = pd.read_csv("/content/drive/MyDrive/FODS__FP/Dataset/subscriptions_60000.csv", low_memory=False)
transDf = pd.read_csv("/content/drive/MyDrive/FODS__FP/Dataset/transactions_600000.csv", low_memory=False)

len(subsDf.axes[0]), len(transDf.axes[0])

(56703, 510867)

## Data Sample

In [3]:
subsDf.head()

,id,created_at,country,operator,service,source,msisdn,status,cycle,adnet,...,service_price,currency,profile_status,publisher,trxid,pixel,handset,browser,attempt_charging,success_billing
0,4772183,2023-11-15 01:24:39.890 +0700,ID,id-telkomsel-mks,SLYPEE FSC,NaN,SHDC-UIMZr0MUR79HfOsXdjxJBLpYg KxaRHmtGxtc/05qWQ,0,daily,NaN,...,1000,NaN,NaN,NaN,1.702232400288E+22,NaN,NaN,NaN,1,1
1,9850460,2023-11-15 22:53:58.779 +0700,ID,telesatpass,PLAYZONE,sms,SHDC-SE2/sv 8CjW4dwFHjeQrdAaJah6g8F1F mS1LpmkyJQ=,1,daily,adg,...,0,IDR,Active,NaN,2.0231211067191E+24,NaN,NaN,NaN,35,4
2,16139146,2023-11-21 12:28:28.178 +0700,ID,id-telkomsel-mks,SLYPEE DFS,NaN,SHDC-yqSKHCM7crgLi6E45ffYl 1X/6kaAjoFqG 77XO0vd8,1,daily,NaN,...,1000,NaN,NaN,NaN,10885399605,NaN,NaN,NaN,6,0
3,58769,2023-10-24 02:25:37.089 +0700,ID,telkomsel,CLOUDPLAY,WAP,SHDC-n21EFe9DWJPzG/jfTc6e+bXpYIcOgMtQTezeMkaxzXM=,1,2d,adn,...,2000,IDR,active,NaN,4e28fd8b-15ab-402c-971f-f786a4e9f52f,NaN,NaN,NaN,1,1
4,10936057,2023-11-16 01:54:53.428 +0700,ID,telkomsel,CLOUDPLAY,WAP,SHDC-DHR7Jy5mhF4P8zX8lyIHQQ6wb4Kdd7XJDD+MBRpCkB0=,1,2d,adn,...,2000,IDR,active,NaN,09737f4a-a536-458d-9411-a5be609aacd3,NaN,NaN,NaN,1,0


In [4]:
transDf.head()

,id,created_at,country,operator,service,source,msisdn,event,event_date,cycle,...,currency,charge_date,publisher,handset,browser,trxid,telco_api_url,telco_api_response,sms_content,status_sms
0,28852369,2023-10-28 02:15:37.361 +0700,ID,telkomsel,CLOUDPLAY,WAP,SHDC-5G7OGPEO6MzJ7nwYXirYcLOogI5VqBHOAq5+42DyGqU=,RENEWAL,2023-10-27 15:23:37.000 +0700,2d,...,IDR,2023-10-27 15:23:37.000 +0700,NaN,NaN,NaN,6dcd7ae2-cb63-4f84-a246-38a1ff82bfb1,https://api.digitalcore.telkomsel.com,03:03:21 AM,[2220] Cloudplay Games bisa kamu mainkan sekar...,FAILED
1,28852386,2023-10-28 02:15:37.371 +0700,ID,telkomsel,CLOUDPLAY,WAP,SHDC-hDVko39dST8Ah8SVzhdmsVKP4U0zw+a/z/aGctqVU/U=,RENEWAL,2023-10-27 15:23:36.000 +0700,2d,...,IDR,2023-10-27 15:23:36.000 +0700,NaN,NaN,NaN,d39dccaf-a079-4db8-9b38-ee3ab7104151,https://api.digitalcore.telkomsel.com,03:03:21 AM,[2220] Cloudplay Games bisa kamu mainkan sekar...,FAILED
2,28852404,2023-10-28 02:15:37.382 +0700,ID,telkomsel,CLOUDPLAY,WAP,SHDC-Mw+Q/dlHqvfIdQj8sw1vPrvpYJJsQO5bWBGoHiaD5r0=,RENEWAL,2023-10-27 15:23:37.000 +0700,2d,...,IDR,2023-10-27 15:23:37.000 +0700,NaN,NaN,NaN,b5341ae1-bde0-4fc0-a380-22e8d17df403,https://api.digitalcore.telkomsel.com,03:03:27 AM,[2220] Cloudplay Games bisa kamu mainkan sekar...,FAILED
3,28852420,2023-10-28 02:15:37.393 +0700,ID,telkomsel,CLOUDPLAY,WAP,SHDC-MwgrjUbHwET57+lFyBkiRNlt+Z4VCwbk8k94TC2zB7Q=,RENEWAL,2023-10-27 15:23:38.000 +0700,2d,...,IDR,2023-10-27 15:23:38.000 +0700,NaN,NaN,NaN,36e0233d-7bb3-4e26-9040-7a68a63580cc,https://api.digitalcore.telkomsel.com,03:03:21 AM,[2220] Cloudplay Games bisa kamu mainkan sekar...,FAILED
4,28852439,2023-10-28 02:15:37.405 +0700,ID,telkomsel,CLOUDPLAY,WAP,SHDC-euAScN3tA0PlYfv4oqIwDC4I5UOAlpEe4qXnGHfh7Ns=,RENEWAL,2023-10-27 15:23:38.000 +0700,2d,...,IDR,2023-10-27 15:23:38.000 +0700,NaN,NaN,NaN,3ccd6083-e609-4521-ae58-d989cbd4184c,https://api.digitalcore.telkomsel.com,03:03:21 AM,[2220] Cloudplay Games bisa kamu mainkan sekar...,FAILED


## Data Cleaning

In [5]:
subsDf.isna().sum()

id                       0
created_at               0
country                  0
operator                 0
service                  0
source               10644
msisdn                   0
status                   0
cycle                    1
adnet                26938
revenue                  0
subs_date                0
renewal_date            33
freemium_end_date    35527
unsubs_from          31174
unsubs_date          23198
service_price            0
currency             12744
profile_status       17602
publisher            34339
trxid                23735
pixel                44687
handset              43752
browser              37664
attempt_charging         0
success_billing          0
dtype: int64

In [6]:
subsDf = subsDf.drop(columns=['trxid', 'pixel', 'handset', 'browser', 'id'])

In [7]:
subsDf['source'] = subsDf['source'].replace({'WAP': 'wap', 'SMS': 'sms'})

In [8]:
subsDf['unsubs_from'] = subsDf['unsubs_from'].replace({'WAP': 'wap', 'SMS': 'sms'})

In [9]:
subsDf['status'] = subsDf['status'].replace({2: 1})

In [10]:
subsDf['service'] = subsDf['service'].replace({'SLYPEE FSC': 'slypee', 'SLYPEE DFS': 'slypee', 'slypee fsc': 'slypee', 'slypee dfs': 'slypee', 'CLOUDPLAY 1': 'CLOUDPLAY'})

In [11]:
subsDf['publisher'] = subsDf['publisher'].replace({'LIG': 'lig',
                                                   'BTM': 'btm',
                                                   'STAR': 'star',
                                                   'MOV': 'mov',
                                                   'LIG': 'lig',
                                                   'VAD': 'vad',
                                                   'CAD': 'cad',
                                                   'ATN': 'atn',
                                                   'PRT': 'prt'})

subsDf['publisher'] = subsDf['publisher'].apply(lambda x: 'star' if pd.notna(x) and 'star' in x.lower() else x)
subsDf['publisher'] = subsDf['publisher'].apply(lambda x: 'lig' if pd.notna(x) and 'lig' in x.lower() else x)

In [12]:
subsDf['operator'] = subsDf['operator'].apply(lambda x: 'telkomsel' if pd.notna(x) and 'telkomsel' in x.lower() else x)

In [13]:
subsDf['adnet'] = subsDf['adnet'].apply(lambda x: 'star' if pd.notna(x) and 'star' in x.lower() else x)
subsDf['adnet'] = subsDf['adnet'].apply(lambda x: 'lig' if pd.notna(x) and 'lig' in x.lower() else x)
subsDf['adnet'] = subsDf['adnet'].apply(lambda x: 'mvls' if pd.notna(x) and 'mvls' in x.lower() else x)
subsDf['adnet'] = subsDf['adnet'].apply(lambda x: 'mobip' if pd.notna(x) and 'mobip' in x.lower() else x)
subsDf['adnet'] = subsDf['adnet'].apply(lambda x: 'adn' if pd.notna(x) and 'adn' in x.lower() else x)
subsDf['adnet'] = subsDf['adnet'].apply(lambda x: str(x))
subsDf['adnet'] = subsDf['adnet'].apply(lambda x: x.lower())

In [14]:
subsDf['source'] = subsDf['source'].replace({'USSD_W': 'USSD', 'USSD_D': 'USSD', 'mo_ussd': 'USSD', 'mo_sms': 'sms'})
subsDf['unsubs_from'] = subsDf['unsubs_from'].replace({'mo_ussd': 'USSD', 'mo_sms': 'sms'})

In [15]:
subsDf[['source', 'adnet', 'publisher']] = subsDf[['source', 'adnet', 'publisher']].fillna('Unknown')

In [16]:
subsDf.loc[(subsDf['country'] == 'ID') & (subsDf['currency'].isna()), 'currency'] = 'IDR'
subsDf.loc[(subsDf['country'] == 'SN') & (subsDf['currency'].isna()), 'currency'] = 'XOF'
subsDf['currency'] = subsDf['currency'].replace({'Kip': 'LAK', 'BATH': 'THB', 'BAHT': 'THB'})

In [17]:
# Indonesian Rupiah (IDR) exchange rate retrieved from Forbes Advisor (27/12/2023 13:56 UTC)

def convert_revenue(row):
    if row['currency'] == 'IDR':
        return float(row['revenue']) * 1
    elif row['currency'] == 'THB':
        return float(row['revenue']) * 447.49
    elif row['currency'] == 'Rial':
        return float(row['revenue']) * 39984.067
    elif row['currency'] == 'Baisa':
        return float(row['revenue']) * 39.984067
    elif row['currency'] == 'LAK':
        return float(row['revenue']) * 0.75
    elif row['currency'] == 'XOF':
        return float(row['revenue']) * 25.986
    elif row['currency'] == 'PHP':
        return float(row['revenue']) * 276.896

subsDf['revenue_rupiah'] = subsDf.apply(convert_revenue, axis=1)
subsDf['revenue_rupiah']

0        1000.0
1        8000.0
2           0.0
3        2000.0
4           0.0
          ...  
56698       0.0
56699       0.0
56700       0.0
56701       0.0
56702       0.0
Name: revenue_rupiah, Length: 56703, dtype: float64

In [18]:
subsDf = subsDf.drop(columns='revenue')

In [19]:
subsDf.dropna(subset=['cycle'], inplace=True)

In [20]:
subsDf['subs_date'] = subsDf['subs_date'].str.replace(r'\s+\+\d{4}', '', regex=True)
subsDf['unsubs_date'] = subsDf['unsubs_date'].str.replace(r'\s+\+\d{4}', '', regex=True)
subsDf['freemium_end_date'] = subsDf['freemium_end_date'].str.replace(r'\s+\+\d{4}', '', regex=True)

In [21]:
def convert_date(date_str):
    try:
        return pd.to_datetime(date_str, format='%d/%m/%Y %H:%M').strftime('%Y-%m-%d %H:%M:%S')
    except ValueError:
        try:
            return pd.to_datetime(date_str).strftime('%Y-%m-%d %H:%M:%S')
        except:
            return date_str

In [22]:
subsDf['subs_date'] = subsDf['subs_date'].apply(convert_date)
subsDf['unsubs_date'] = subsDf['unsubs_date'].apply(convert_date)

In [23]:
columns_to_exclude = ['created_at', 'trxid', 'id', 'msisdn', 'freemium_end_date', 'unsubs_date', 'subs_date', 'renewal_date', 'attempt_charging', 'success_billing', 'revenue', 'revenue_rupiah']
columns_to_include = [col for col in subsDf.columns if col not in columns_to_exclude and 'date' not in col.lower()]

unique_values_dict = {col: subsDf[col].unique() for col in columns_to_include}

unique_values_dict

{'country': array(['ID', 'LA', 'OM', 'PH', 'SN', 'TH'], dtype=object),
 'operator': array(['telkomsel', 'telesatpass', 'smartfren', 'ltc', 'tplus', 'etl',
        'omantel', 'ooredo', 'linguisto', 'smart', 'sen-orange-mtarget',
        'aisgemezz', 'ais'], dtype=object),
 'service': array(['slypee', 'PLAYZONE', 'CLOUDPLAY', 'GAMESPOT', 'FUN1', 'QUIZPRO',
        'GAMESC LKT', 'GAZY', 'GMSPAZE', 'GALAYS', 'gemezz', 'quizy',
        'halogame', 'goaly', 'linguisto', 'omantel', '10291', '10292',
        '9109', 'GE', '459509902', '459505105', '459505104', '459505106',
        '459509903', '459509901', 'GEE', 'GED', 'GEA', 'GE7', 'GEF',
        'GE7A', 'GEC', 'GEB', 'GE1'], dtype=object),
 'source': array(['Unknown', 'sms', 'wap', 'cp1', 'lp3', 'USSD', 'portal', 'lp6',
        'lp7', 'default', 'web'], dtype=object),
 'status': array([ 0,  1, -1]),
 'cycle': array(['daily', '2d', '3d', 'weekly', 'monthly'], dtype=object),
 'adnet': array(['nan', 'adg', 'adn', 'soy', 'prt', 'star', 'mbp', '

In [24]:
subsDf['subs_date'] = pd.to_datetime(subsDf['subs_date'], errors='coerce')
subsDf['subs_date'].min(), subsDf['subs_date'].max()

(Timestamp('2015-02-26 20:18:40'), Timestamp('2024-07-09 23:59:00'))

## Calculate CLV

In [25]:
forClv = subsDf.copy()

In [26]:
forClv['unsubs_date'] = forClv['unsubs_date'].apply(lambda x: '2023-12-12' if pd.notna(x) and '9999' in x.lower() else x)
forClv['unsubs_date'] = forClv['unsubs_date'].apply(lambda x: '2023-12-12' if pd.notna(x) and '0001' in x.lower() else x)

In [27]:
forClv['unsubs_date'] = pd.to_datetime(forClv['unsubs_date'], errors='coerce')
ongoing_subs_date = pd.Timestamp('2023-12-12')
forClv['unsubs_date'].fillna(ongoing_subs_date, inplace=True)

forClv.loc[forClv['unsubs_date'] < forClv['subs_date'], 'unsubs_date'] = ongoing_subs_date

forClv['subscription_period_days'] = (forClv['unsubs_date'] - forClv['subs_date']).dt.days

forClv['subscription_period_days'] = forClv['subscription_period_days'].replace(0, 1)

forClv = forClv.groupby('msisdn').agg({
    'subscription_period_days': 'max',
    'revenue_rupiah': 'sum',
    'country': 'first',
    'operator': lambda x: ', '.join(x.unique()),
    'service': lambda x: ', '.join(x.unique()),
    'source': 'first',
    'status': 'first',
    'cycle': 'first',
    'adnet': lambda x: ', '.join(x.dropna().unique()),
    'subs_date': 'first',
    'unsubs_from': 'first',
    'unsubs_date': 'first',
    'service_price': 'first',
    'currency': 'first',
    'publisher': lambda x: ', '.join(x.dropna().unique()),
    'attempt_charging': 'sum',
    'success_billing': 'sum'
}).reset_index()

# Recalculate CLV
forClv['CLV'] = forClv['revenue_rupiah'] / forClv['subscription_period_days']

forClv['revenue_rupiah'].value_counts()

revenue_rupiah
0.000000e+00    34861
1.000000e+03     5856
3.897900e+03     1061
2.000000e+03      787
1.199522e+07      669
                ...  
5.517801e+05        1
1.259498e+06        1
3.130752e+06        1
1.115555e+06        1
2.800000e+04        1
Name: count, Length: 660, dtype: int64

In [28]:
forClv['CLV'] = forClv['CLV'].replace({-2000: 2000})

In [29]:
conversion_rate = 0.000064
forClv['revenue_usd'] = forClv['revenue_rupiah'] * conversion_rate

forClv[['revenue_rupiah', 'revenue_usd']].head()

,revenue_rupiah,revenue_usd
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0


In [30]:
clv_min = forClv['CLV'].min()
clv_max = forClv['CLV'].max()

def segment_clv(clv):
    if clv_min <= clv <= 1000:
        return 'Very Low'
    elif 1001 <= clv <= 10000:
        return 'Low'
    elif 10001 <= clv <= 100000:
        return 'Medium'
    elif 100001 <= clv <= clv_max:
        return 'High'

forClv['Segment'] = forClv['CLV'].apply(segment_clv)

forClv[['CLV', 'Segment']].head()

,CLV,Segment
0,0.0,Very Low
1,0.0,Very Low
2,0.0,Very Low
3,0.0,Very Low
4,0.0,Very Low


In [31]:
forClv.operator.value_counts()

operator
smart                     10000
omantel                    8752
telkomsel                  7935
sen-orange-mtarget         6703
ais                        6215
tplus                      3719
aisgemezz                  3697
etl                        3233
ltc                        3031
telesatpass                1175
smartfren                   881
ooredo                      754
linguisto                   328
linguisto, omantel           28
omantel, linguisto           28
aisgemezz, ais               14
ais, aisgemezz                6
telkomsel, telesatpass        1
Name: count, dtype: int64

In [32]:
forClv.isna().sum()

msisdn                          0
subscription_period_days     5531
revenue_rupiah                  0
country                         0
operator                        0
service                         0
source                          0
status                          0
cycle                           0
adnet                           0
subs_date                    5531
unsubs_from                 30984
unsubs_date                     0
service_price                   0
currency                        0
publisher                       0
attempt_charging                0
success_billing                 0
CLV                          5531
revenue_usd                     0
Segment                      5531
dtype: int64

In [33]:
very_low_df = forClv[forClv['Segment'] == 'Very Low']
low_df = forClv[forClv['Segment'] == 'Low']
medium_df = forClv[forClv['Segment'] == 'Medium']
high_df = forClv[forClv['Segment'] == 'High']
very_high_df = forClv[forClv['Segment'] == 'Very High']

## Very Low

In [34]:
very_low_df

,msisdn,subscription_period_days,revenue_rupiah,country,operator,service,source,status,cycle,adnet,...,unsubs_from,unsubs_date,service_price,currency,publisher,attempt_charging,success_billing,CLV,revenue_usd,Segment
0,2020008781,44.0,0.0,LA,etl,slypee,lp3,-1,daily,lig,...,cstool,2023-12-12,2000,LAK,lumos,16,0,0.00,0.000,Very Low
1,2020010903,45.0,0.0,LA,etl,slypee,lp3,-1,daily,lig,...,cstool,2023-12-12,2000,LAK,lumos,18,0,0.00,0.000,Very Low
2,2020023548,44.0,0.0,LA,etl,slypee,lp3,-1,daily,lig,...,cstool,2023-12-12,2000,LAK,lumos,16,0,0.00,0.000,Very Low
3,2020025070,44.0,0.0,LA,etl,slypee,lp3,-1,daily,lig,...,cstool,2023-12-12,2000,LAK,lumos,16,0,0.00,0.000,Very Low
4,2020031236,45.0,0.0,LA,etl,slypee,lp3,-1,daily,lig,...,cstool,2023-12-12,2000,LAK,lumos,18,0,0.00,0.000,Very Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56495,[MCV]zazcmbTB61O1uSXu6qR3JA==,32.0,1000.0,ID,smartfren,FUN1,Unknown,0,daily,nan,...,sms,2023-12-12,1000,IDR,Unknown,1,1,31.25,0.064,Very Low
56496,[MCV]zd9woj036A8m2AbWYkIa7g==,32.0,1000.0,ID,smartfren,FUN1,Unknown,0,daily,nan,...,sms,2023-12-12,1000,IDR,Unknown,1,1,31.25,0.064,Very Low
56497,[MCV]zhEIzueBLiPjg/QAgzfvGw==,32.0,1000.0,ID,smartfren,FUN1,Unknown,0,daily,nan,...,sms,2023-12-12,1000,IDR,Unknown,2,1,31.25,0.064,Very Low
56498,[MCV]zj7dxevMx0XVBHNPkl1jtA==,32.0,1000.0,ID,smartfren,FUN1,Unknown,0,daily,nan,...,sms,2023-12-12,1000,IDR,Unknown,2,1,31.25,0.064,Very Low


In [35]:
very_low_expanded = very_low_df.assign(service=very_low_df['service'].str.split(', ')).explode('service')

user_service_matrix = pd.pivot_table(very_low_expanded, index='msisdn', columns='service', aggfunc='size', fill_value=0)

user_service_matrix.shape, user_service_matrix.head()

((47626, 34),
 service     10291  10292  459505104  459505105  459505106  459509901  \
 msisdn                                                                 
 2020008781      0      0          0          0          0          0   
 2020010903      0      0          0          0          0          0   
 2020023548      0      0          0          0          0          0   
 2020025070      0      0          0          0          0          0   
 2020031236      0      0          0          0          0          0   
 
 service     459509902  459509903  9109  CLOUDPLAY  ...  GMSPAZE  PLAYZONE  \
 msisdn                                             ...                      
 2020008781          0          0     0          0  ...        0         0   
 2020010903          0          0     0          0  ...        0         0   
 2020023548          0          0     0          0  ...        0         0   
 2020025070          0          0     0          0  ...        0         0   
 2020

### COSINE SIMILARITY

In [36]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(user_service_matrix.T)

cosine_sim_df = pd.DataFrame(cosine_sim, index=user_service_matrix.columns, columns=user_service_matrix.columns)

cosine_sim_df.head()

service,10291,10292,459505104,459505105,459505106,459509901,459509902,459509903,9109,CLOUDPLAY,...,GMSPAZE,PLAYZONE,QUIZPRO,gemezz,goaly,halogame,linguisto,omantel,quizy,slypee
service,,,,,,,,,,,,,,,,,,,,,
10291,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10292,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
459505104,0.0,0.0,1.000000,0.007731,0.005175,0.006360,0.006462,0.004067,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
459505105,0.0,0.0,0.007731,1.000000,0.002151,0.002643,0.002014,0.002535,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
459505106,0.0,0.0,0.005175,0.002151,1.000000,0.007076,0.005393,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
def recommend_services_cosine(user_id, user_service_matrix, similarity_matrix, top_n=2):
    subscribed_services = user_service_matrix.loc[user_id]
    subscribed_services = subscribed_services[subscribed_services > 0].index.tolist()

    service_similarities = similarity_matrix[subscribed_services].mean(axis=1)

    for service in subscribed_services:
        service_similarities.drop(service, inplace=True, errors='ignore')

    recommended_services = service_similarities.nlargest(top_n).index.tolist()

    return recommended_services

sample_users = very_low_df['msisdn'].sample(n=10, random_state=1)

recommendations = {}
for user in sample_users:
    recommended_services = recommend_services_cosine(user, user_service_matrix, cosine_sim_df)
    recommendations[user] = recommended_services

recommendations

{'66927283010': ['459509901', '459509902'],
 '2078536652': ['halogame', 'quizy'],
 '96890961474': ['linguisto', 'omantel'],
 '2078600815': ['halogame', 'quizy'],
 '66817595537': ['459505104', '459505105'],
 '96891473528': ['linguisto', 'omantel'],
 '639102048395': ['10292', '459505104'],
 '639129058204': ['10292', '459505104'],
 'SHDC-F87reHqX+06xd4kA2Hc50BRj0pr8ZHQwlhMXaMw2VvI=': ['QUIZPRO', '10291'],
 'SHDC-MdV5t9dCZaPAk8Gom322tRjFY9uKfqajxDYpMF2eqow': ['10291', '10292']}

In [38]:
very_low_cosine = very_low_df.copy()

very_low_cosine['Recommended_Services'] = very_low_cosine['msisdn'].apply(lambda x: recommend_services_cosine(x, user_service_matrix, cosine_sim_df))

very_low_cosine[['msisdn', 'service', 'Recommended_Services']].head()

,msisdn,service,Recommended_Services
0,2020008781,slypee,"[10291, 10292]"
1,2020010903,slypee,"[10291, 10292]"
2,2020023548,slypee,"[10291, 10292]"
3,2020025070,slypee,"[10291, 10292]"
4,2020031236,slypee,"[10291, 10292]"


In [39]:
very_low_cosine['Recommended_Services'].value_counts()

Recommended_Services
[halogame, quizy]         12710
[10291, 10292]             9985
[10292, 459505104]         8032
[linguisto, omantel]       4986
[QUIZPRO, 10291]           2705
[10291, 459505104]         1950
[459505104, 459505105]     1309
[gemezz, halogame]         1081
[GAMESPOT, CLOUDPLAY]       688
[omantel, goaly]            658
[459505105, 459509902]      629
[459509901, 459505104]      581
[459509901, 459509902]      515
[459505104, GE]             404
[459509902, 459505104]      373
[459505106, 459509902]      334
[gemezz, quizy]             326
[linguisto, goaly]          260
[omantel, 10291]             37
[goaly, 10291]               11
[459505104, 459505106]       11
[linguisto, 10291]            8
[halogame, 10291]             8
[quizy, 10291]                5
[459509901, 459505106]        4
[459505104, 459509902]        4
[459505104, 459509901]        4
[459509902, 459505106]        3
[459509902, 459505105]        2
[gemezz, 10291]               1
[CLOUDPLAY, 10291] 

### Very Low Cosine Metrics Test

In [40]:
very_low_cosine_test = very_low_cosine[very_low_cosine['service'].str.contains(',')]

In [41]:
import random

very_low_cosine_test['actual_services'] = very_low_cosine_test['service']

very_low_cosine_test['service'] = very_low_cosine_test['service'].apply(lambda x: random.choice(x.split(', ')))
# very_low_cosine_test['service'] = very_low_cosine_test['service'].apply(lambda x: x.split(', ')[0])

very_low_cosine_test[['msisdn', 'service', 'actual_services']]

,msisdn,service,actual_services
3406,2052241312,quizy,"gemezz, quizy"
3437,2052308156,gemezz,"gemezz, quizy"
3440,2052317347,quizy,"quizy, gemezz"
3499,2052474283,gemezz,"gemezz, quizy"
3639,2052824903,gemezz,"quizy, gemezz"
...,...,...,...
38837,96899369161,linguisto,"linguisto, goaly"
39002,96899491483,goaly,"omantel, goaly"
39193,96899583406,goaly,"goaly, omantel, linguisto"
47283,SHDC-3QlPeGo6p0A22taJsTa4W3FoibkmgW9IgpEsS3ThPz4=,GAMESPOT,"QUIZPRO, GAMESPOT"


In [42]:
filtered_df = very_low_cosine_test[very_low_cosine_test['actual_services'].str.count(',') == 2]

result_df = filtered_df[['msisdn', 'service', 'actual_services']]

In [43]:
very_low_cosine_test[['msisdn', 'service', 'actual_services']]

,msisdn,service,actual_services
3406,2052241312,quizy,"gemezz, quizy"
3437,2052308156,gemezz,"gemezz, quizy"
3440,2052317347,quizy,"quizy, gemezz"
3499,2052474283,gemezz,"gemezz, quizy"
3639,2052824903,gemezz,"quizy, gemezz"
...,...,...,...
38837,96899369161,linguisto,"linguisto, goaly"
39002,96899491483,goaly,"omantel, goaly"
39193,96899583406,goaly,"goaly, omantel, linguisto"
47283,SHDC-3QlPeGo6p0A22taJsTa4W3FoibkmgW9IgpEsS3ThPz4=,GAMESPOT,"QUIZPRO, GAMESPOT"


In [44]:
very_low_cosine_test_matrix = pd.pivot_table(very_low_cosine_test, index='msisdn', columns='service', aggfunc='size', fill_value=0)

very_low_cosine_test['Recommended_Services'] = very_low_cosine_test['msisdn'].apply(lambda x: recommend_services_cosine(x, very_low_cosine_test_matrix, cosine_sim_df))

very_low_cosine_test[['msisdn', 'service', 'actual_services', 'Recommended_Services']].head()

,msisdn,service,actual_services,Recommended_Services
3406,2052241312,quizy,"gemezz, quizy","[gemezz, halogame]"
3437,2052308156,gemezz,"gemezz, quizy","[halogame, quizy]"
3440,2052317347,quizy,"quizy, gemezz","[gemezz, halogame]"
3499,2052474283,gemezz,"gemezz, quizy","[halogame, quizy]"
3639,2052824903,gemezz,"quizy, gemezz","[halogame, quizy]"


In [45]:
very_low_cosine_test['Recommended_Services'] = very_low_cosine_test['Recommended_Services'].astype(str).str.replace("[\[\]' ]", '', regex=True).str.split(',')
very_low_cosine_test['actual_services'] = very_low_cosine_test['actual_services'].astype(str).str.replace(' ', '').str.split(',')
very_low_cosine_test['service'] = very_low_cosine_test['service'].astype(str).str.split(',')

In [46]:
def calculate_precision(recommended, actual):
    recommended_set = set(recommended)
    actual_set = set(actual)
    precision = 1 if len(recommended_set.intersection(actual_set)) > 0 else 0
    return precision

def calculate_recall(recommended, actual, service):
    relevant_actual_services = set(actual) - set(service)

    if not relevant_actual_services:
        return None

    recommended_set = set(recommended)
    correct_recommendations = recommended_set.intersection(relevant_actual_services)
    recall = len(correct_recommendations) / len(relevant_actual_services)

    return recall

In [47]:
very_low_cosine_test['precision'] = very_low_cosine_test.apply(lambda row: calculate_precision(row['Recommended_Services'], row['actual_services']), axis=1)
very_low_cosine_test['recall'] = very_low_cosine_test.apply(lambda row: calculate_recall(row['Recommended_Services'], row['actual_services'], row['service']), axis=1)

average_cosine_precision = very_low_cosine_test['precision'].mean()
average_cosine_recall = very_low_cosine_test['recall'].mean()

average_cosine_precision, average_cosine_recall

(0.8305084745762712, 0.8305084745762712)

In [48]:
very_low_cosine_test['target_services'] = very_low_cosine_test.apply(lambda row: [service for service in row['actual_services'] if service not in row['service']], axis=1)

# Calculating True Positive (TP), False Positive (FP), False Negative (FN), and True Negative (TN)
def calculate_metrics(row):
    tp = len(set(row['target_services']) & set(row['Recommended_Services']))
    fp = 3 - tp
    fn = len(row['actual_services']) - tp
    tn = 34 - (tp + fp + fn)  # Total services (34) minus (TP + FP + FN)
    accuracy = (tp + tn) / 34
    return pd.Series([tp, fp, fn, tn, accuracy])

very_low_cosine_test[['TP', 'FP', 'FN', 'TN', 'Accuracy']] = very_low_cosine_test.apply(calculate_metrics, axis=1)

average_accuracy = very_low_cosine_test['Accuracy'].mean()

average_accuracy

0.9020438683948154

### JACCARD

In [49]:
from sklearn.metrics import pairwise_distances

very_low_expanded = very_low_df.assign(service=very_low_df['service'].str.split(', ')).explode('service')
user_service_matrix = pd.pivot_table(very_low_expanded, index='msisdn', columns='service', aggfunc='size', fill_value=0)

jaccard_sim = 1 - pairwise_distances(user_service_matrix.T.values, metric="jaccard")

jaccard_sim_df = pd.DataFrame(jaccard_sim, index=user_service_matrix.columns, columns=user_service_matrix.columns)

jaccard_sim_df.head()

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


service,10291,10292,459505104,459505105,459505106,459509901,459509902,459509903,9109,CLOUDPLAY,...,GMSPAZE,PLAYZONE,QUIZPRO,gemezz,goaly,halogame,linguisto,omantel,quizy,slypee
service,,,,,,,,,,,,,,,,,,,,,
10291,1.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10292,0.0,1.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
459505104,0.0,0.0,1.000000,0.003788,0.00258,0.003040,0.003239,0.001965,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
459505105,0.0,0.0,0.003788,1.000000,0.00107,0.001318,0.000992,0.001267,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
459505106,0.0,0.0,0.002580,0.001070,1.00000,0.003476,0.002698,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
def recommend_services_jaccard(user_id, user_service_matrix, similarity_matrix, top_n=2):
    subscribed_services = user_service_matrix.loc[user_id]
    subscribed_services = subscribed_services[subscribed_services > 0].index.tolist()

    service_similarities = similarity_matrix[subscribed_services].mean(axis=1)

    for service in subscribed_services:
        service_similarities.drop(service, inplace=True, errors='ignore')

    recommended_services = service_similarities.nlargest(top_n).index.tolist()

    return recommended_services

sample_users = very_low_df['msisdn'].sample(n=10, random_state=1)

recommendations = {}
for user in sample_users:
    recommended_services = recommend_services_jaccard(user, user_service_matrix, jaccard_sim_df)
    recommendations[user] = recommended_services

recommendations

{'66927283010': ['459509901', '459509902'],
 '2078536652': ['quizy', 'halogame'],
 '96890961474': ['linguisto', 'omantel'],
 '2078600815': ['quizy', 'halogame'],
 '66817595537': ['459505104', '459505105'],
 '96891473528': ['linguisto', 'omantel'],
 '639102048395': ['10292', '459505104'],
 '639129058204': ['10292', '459505104'],
 'SHDC-F87reHqX+06xd4kA2Hc50BRj0pr8ZHQwlhMXaMw2VvI=': ['QUIZPRO', '10291'],
 'SHDC-MdV5t9dCZaPAk8Gom322tRjFY9uKfqajxDYpMF2eqow': ['10291', '10292']}

In [51]:
very_low_jaccard = very_low_df.copy()

very_low_jaccard['Recommended_Services'] = very_low_jaccard['msisdn'].apply(lambda x: recommend_services_jaccard(x, user_service_matrix, jaccard_sim_df))

very_low_jaccard[['msisdn', 'service', 'Recommended_Services']]

,msisdn,service,Recommended_Services
0,2020008781,slypee,"[10291, 10292]"
1,2020010903,slypee,"[10291, 10292]"
2,2020023548,slypee,"[10291, 10292]"
3,2020025070,slypee,"[10291, 10292]"
4,2020031236,slypee,"[10291, 10292]"
...,...,...,...
56495,[MCV]zazcmbTB61O1uSXu6qR3JA==,FUN1,"[10291, 10292]"
56496,[MCV]zd9woj036A8m2AbWYkIa7g==,FUN1,"[10291, 10292]"
56497,[MCV]zhEIzueBLiPjg/QAgzfvGw==,FUN1,"[10291, 10292]"
56498,[MCV]zj7dxevMx0XVBHNPkl1jtA==,FUN1,"[10291, 10292]"


In [52]:
very_low_jaccard['Recommended_Services'].value_counts()

Recommended_Services
[quizy, halogame]         12710
[10291, 10292]             9985
[10292, 459505104]         8032
[linguisto, omantel]       4986
[QUIZPRO, 10291]           2705
[10291, 459505104]         1950
[459505104, 459505105]     1309
[halogame, gemezz]         1081
[GAMESPOT, CLOUDPLAY]       688
[omantel, goaly]            658
[459505105, 459509902]      629
[459505104, 459509901]      582
[459509901, 459509902]      515
[459505104, GE]             404
[459509902, 459505104]      373
[459505106, 459509902]      334
[quizy, gemezz]             326
[linguisto, goaly]          260
[omantel, 10291]             37
[goaly, 10291]               11
[459505104, 459505106]       10
[halogame, 10291]             8
[linguisto, 10291]            8
[quizy, 10291]                5
[459505104, 459509902]        4
[459509901, 459505106]        4
[459509901, 459505104]        3
[459509902, 459505106]        3
[459509902, 459505105]        2
[459505106, 459505104]        1
[gemezz, 10291]    

### Very Low Jaccard Metrics Test

In [53]:
very_low_jaccard_test = very_low_jaccard[very_low_jaccard['service'].str.contains(',')]

In [54]:
very_low_jaccard_test['actual_services'] = very_low_jaccard_test['service']

very_low_jaccard_test['service'] = very_low_jaccard_test['service'].apply(lambda x: random.choice(x.split(', ')))
# very_low_jaccard_test['service'] = very_low_jaccard_test['service'].apply(lambda x: x.split(', ')[0])

very_low_jaccard_test[['msisdn', 'service', 'actual_services']]

,msisdn,service,actual_services
3406,2052241312,quizy,"gemezz, quizy"
3437,2052308156,quizy,"gemezz, quizy"
3440,2052317347,quizy,"quizy, gemezz"
3499,2052474283,gemezz,"gemezz, quizy"
3639,2052824903,quizy,"quizy, gemezz"
...,...,...,...
38837,96899369161,linguisto,"linguisto, goaly"
39002,96899491483,omantel,"omantel, goaly"
39193,96899583406,omantel,"goaly, omantel, linguisto"
47283,SHDC-3QlPeGo6p0A22taJsTa4W3FoibkmgW9IgpEsS3ThPz4=,QUIZPRO,"QUIZPRO, GAMESPOT"


In [55]:
very_low_jaccard_test_matrix = pd.pivot_table(very_low_jaccard_test, index='msisdn', columns='service', aggfunc='size', fill_value=0)

very_low_jaccard_test['Recommended_Services'] = very_low_jaccard_test['msisdn'].apply(lambda x: recommend_services_jaccard(x, very_low_jaccard_test_matrix, jaccard_sim_df))

very_low_jaccard_test[['msisdn', 'service', 'actual_services', 'Recommended_Services']].head()

,msisdn,service,actual_services,Recommended_Services
3406,2052241312,quizy,"gemezz, quizy","[halogame, gemezz]"
3437,2052308156,quizy,"gemezz, quizy","[halogame, gemezz]"
3440,2052317347,quizy,"quizy, gemezz","[halogame, gemezz]"
3499,2052474283,gemezz,"gemezz, quizy","[quizy, halogame]"
3639,2052824903,quizy,"quizy, gemezz","[halogame, gemezz]"


In [56]:
very_low_jaccard_test['Recommended_Services'] = very_low_jaccard_test['Recommended_Services'].astype(str).str.replace("[\[\]' ]", '', regex=True).str.split(',')
very_low_jaccard_test['actual_services'] = very_low_jaccard_test['actual_services'].astype(str).str.replace(' ', '').str.split(',')
very_low_jaccard_test['service'] = very_low_jaccard_test['service'].astype(str).str.split(',')

In [57]:
def calculate_precision(recommended, actual):
    recommended_set = set(recommended)
    actual_set = set(actual)
    precision = 1 if len(recommended_set.intersection(actual_set)) > 0 else 0
    return precision

def calculate_recall(recommended, actual, service):
    relevant_actual_services = set(actual) - set(service)

    if not relevant_actual_services:
        return None

    recommended_set = set(recommended)
    correct_recommendations = recommended_set.intersection(relevant_actual_services)
    recall = len(correct_recommendations) / len(relevant_actual_services)

    return recall

In [58]:
very_low_jaccard_test['precision'] = very_low_jaccard_test.apply(lambda row: calculate_precision(row['Recommended_Services'], row['actual_services']), axis=1)
very_low_jaccard_test['recall'] = very_low_jaccard_test.apply(lambda row: calculate_recall(row['Recommended_Services'], row['actual_services'], row['service']), axis=1)

average_jaccard_precision = very_low_jaccard_test['precision'].mean()
average_jaccard_recall = very_low_jaccard_test['recall'].mean()

average_jaccard_precision, average_jaccard_recall

(0.7966101694915254, 0.7966101694915254)

In [59]:
very_low_jaccard_test['target_services'] = very_low_jaccard_test.apply(lambda row: [service for service in row['actual_services'] if service not in row['service']], axis=1)

def calculate_metrics(row):
    tp = len(set(row['target_services']) & set(row['Recommended_Services']))
    fp = 2 - tp
    fn = len(row['actual_services']) - tp
    tn = 34 - (tp + fp + fn)
    accuracy = (tp + tn) / 34
    return pd.Series([tp, fp, fn, tn, accuracy])

very_low_jaccard_test[['TP', 'FP', 'FN', 'TN', 'Accuracy']] = very_low_jaccard_test.apply(calculate_metrics, axis=1)

average_accuracy = very_low_jaccard_test['Accuracy'].mean()

average_accuracy

0.9294616151545364

### EUCLIDEAN

In [60]:
very_low_expanded = very_low_df.assign(service=very_low_df['service'].str.split(', ')).explode('service')
user_service_matrix = pd.pivot_table(very_low_expanded, index='msisdn', columns='service', aggfunc='size', fill_value=0)

In [61]:
from scipy.spatial.distance import euclidean

euclidean_dist_matrix = pairwise_distances(user_service_matrix.T, metric='euclidean')

euclidean_similarity = 1 / (1 + euclidean_dist_matrix)

euclidean_sim_df = pd.DataFrame(euclidean_similarity, index=user_service_matrix.columns, columns=user_service_matrix.columns)

euclidean_sim_df.head()

service,10291,10292,459505104,459505105,459505106,459509901,459509902,459509903,9109,CLOUDPLAY,...,GMSPAZE,PLAYZONE,QUIZPRO,gemezz,goaly,halogame,linguisto,omantel,quizy,slypee
service,,,,,,,,,,,,,,,,,,,,,
10291,1.000000,0.009910,0.010621,0.010763,0.010697,0.010808,0.010652,0.010789,0.011032,0.009855,...,0.011032,0.010708,0.010594,0.006893,0.008673,0.010816,0.010584,0.010849,0.010362,0.007854
10292,0.009910,1.000000,0.019253,0.020149,0.019720,0.020447,0.019441,0.020318,0.022116,0.015540,...,0.022116,0.019791,0.019091,0.008188,0.011826,0.020504,0.019031,0.020737,0.017814,0.009962
459505104,0.010621,0.019253,1.000000,0.029909,0.028524,0.030894,0.027712,0.030418,0.037743,0.018854,...,0.037743,0.028668,0.026640,0.008575,0.013098,0.031001,0.026476,0.031833,0.023445,0.010685
459505105,0.010763,0.020149,0.029909,1.000000,0.031684,0.035049,0.030550,0.034398,0.046525,0.019693,...,0.046525,0.031950,0.029204,0.008649,0.013368,0.035297,0.028989,0.036546,0.025129,0.010830
459505106,0.010697,0.019720,0.028524,0.031684,1.000000,0.032975,0.029153,0.032328,0.041780,0.019293,...,0.041780,0.030303,0.027933,0.008615,0.013242,0.033106,0.027745,0.034127,0.024309,0.010763


In [62]:
def recommend_services_euclidean(user_id, user_service_matrix, similarity_matrix, top_n=2):
    subscribed_services = user_service_matrix.loc[user_id]
    subscribed_services = subscribed_services[subscribed_services > 0].index.tolist()

    service_similarities = similarity_matrix[subscribed_services].mean(axis=1)

    for service in subscribed_services:
        service_similarities.drop(service, inplace=True, errors='ignore')

    recommended_services = service_similarities.nlargest(top_n).index.tolist()

    return recommended_services

sample_users = very_low_df['msisdn'].sample(n=10, random_state=1)

euclidean_recommendations = {}
for user in sample_users:
    recommended_services = recommend_services_euclidean(user, user_service_matrix, euclidean_sim_df)
    euclidean_recommendations[user] = recommended_services

euclidean_recommendations

{'66927283010': ['GALAYS', '9109'],
 '2078536652': ['GALAYS', '9109'],
 '96890961474': ['GALAYS', '9109'],
 '2078600815': ['GALAYS', '9109'],
 '66817595537': ['GALAYS', '9109'],
 '96891473528': ['GALAYS', '9109'],
 '639102048395': ['GALAYS', '9109'],
 '639129058204': ['GALAYS', '9109'],
 'SHDC-F87reHqX+06xd4kA2Hc50BRj0pr8ZHQwlhMXaMw2VvI=': ['GALAYS', '9109'],
 'SHDC-MdV5t9dCZaPAk8Gom322tRjFY9uKfqajxDYpMF2eqow': ['GALAYS', '9109']}

In [63]:
very_low_euclidean = very_low_df.copy()

very_low_euclidean['Recommended_Services'] = very_low_euclidean['msisdn'].apply(
    lambda x: recommend_services_euclidean(x, user_service_matrix, euclidean_sim_df))

very_low_euclidean[['msisdn', 'service', 'Recommended_Services']].head()

,msisdn,service,Recommended_Services
0,2020008781,slypee,"[GALAYS, 9109]"
1,2020010903,slypee,"[GALAYS, 9109]"
2,2020023548,slypee,"[GALAYS, 9109]"
3,2020025070,slypee,"[GALAYS, 9109]"
4,2020031236,slypee,"[GALAYS, 9109]"


In [64]:
very_low_euclidean['Recommended_Services'].value_counts()

Recommended_Services
[GALAYS, 9109]    47620
[GALAYS, GEC]         5
[9109, GEC]           1
Name: count, dtype: int64

### Very Low Euclidean Metrics Test

In [65]:
very_low_euclidean_test = very_low_euclidean[very_low_euclidean['service'].str.contains(',')]

In [66]:
very_low_euclidean_test['actual_services'] = very_low_euclidean_test['service']

very_low_euclidean_test['service'] = very_low_euclidean_test['service'].apply(lambda x: random.choice(x.split(', ')))
# very_low_euclidean_test['service'] = very_low_euclidean_test['service'].apply(lambda x: x.split(', ')[0])

very_low_euclidean_test[['msisdn', 'service', 'actual_services']]

,msisdn,service,actual_services
3406,2052241312,gemezz,"gemezz, quizy"
3437,2052308156,gemezz,"gemezz, quizy"
3440,2052317347,gemezz,"quizy, gemezz"
3499,2052474283,quizy,"gemezz, quizy"
3639,2052824903,gemezz,"quizy, gemezz"
...,...,...,...
38837,96899369161,linguisto,"linguisto, goaly"
39002,96899491483,omantel,"omantel, goaly"
39193,96899583406,omantel,"goaly, omantel, linguisto"
47283,SHDC-3QlPeGo6p0A22taJsTa4W3FoibkmgW9IgpEsS3ThPz4=,QUIZPRO,"QUIZPRO, GAMESPOT"


In [67]:
very_low_euclidean_test_matrix = pd.pivot_table(very_low_euclidean_test, index='msisdn', columns='service', aggfunc='size', fill_value=0)

very_low_euclidean_test['Recommended_Services'] = very_low_euclidean_test['msisdn'].apply(lambda x: recommend_services_euclidean(x, very_low_euclidean_test_matrix, euclidean_sim_df))

very_low_euclidean_test[['msisdn', 'service', 'actual_services', 'Recommended_Services']].head()

,msisdn,service,actual_services,Recommended_Services
3406,2052241312,gemezz,"gemezz, quizy","[GALAYS, 9109]"
3437,2052308156,gemezz,"gemezz, quizy","[GALAYS, 9109]"
3440,2052317347,gemezz,"quizy, gemezz","[GALAYS, 9109]"
3499,2052474283,quizy,"gemezz, quizy","[GALAYS, 9109]"
3639,2052824903,gemezz,"quizy, gemezz","[GALAYS, 9109]"


In [68]:
very_low_euclidean_test['Recommended_Services'] = very_low_euclidean_test['Recommended_Services'].astype(str).str.replace("[\[\]' ]", '', regex=True).str.split(',')
very_low_euclidean_test['actual_services'] = very_low_euclidean_test['actual_services'].astype(str).str.replace(' ', '').str.split(',')
very_low_euclidean_test['service'] = very_low_euclidean_test['service'].astype(str).str.split(',')

In [69]:
very_low_euclidean_test['precision'] = very_low_euclidean_test.apply(lambda row: calculate_precision(row['Recommended_Services'], row['actual_services']), axis=1)
very_low_euclidean_test['recall'] = very_low_euclidean_test.apply(lambda row: calculate_recall(row['Recommended_Services'], row['actual_services'], row['service']), axis=1)

average_euclidean_precision = very_low_euclidean_test['precision'].mean()
average_euclidean_recall = very_low_euclidean_test['recall'].mean()

average_euclidean_precision, average_euclidean_recall

(0.0, 0.0)

In [70]:
very_low_euclidean_test['target_services'] = very_low_euclidean_test.apply(lambda row: [service for service in row['actual_services'] if service not in row['service']], axis=1)

def calculate_metrics(row):
    tp = len(set(row['target_services']) & set(row['Recommended_Services']))
    fp = 2 - tp
    fn = len(row['actual_services']) - tp
    tn = 34 - (tp + fp + fn)
    accuracy = (tp + tn) / 34
    return pd.Series([tp, fp, fn, tn, accuracy])

very_low_euclidean_test[['TP', 'FP', 'FN', 'TN', 'Accuracy']] = very_low_euclidean_test.apply(calculate_metrics, axis=1)

average_accuracy = very_low_euclidean_test['Accuracy'].mean()

average_accuracy

0.8821036889332005

### MANHATTAN / CITY BLOCK

In [71]:
very_low_expanded = very_low_df.assign(service=very_low_df['service'].str.split(', ')).explode('service')
user_service_matrix = pd.pivot_table(very_low_expanded, index='msisdn', columns='service', aggfunc='size', fill_value=0)

In [72]:
manhattan_dist_matrix = pairwise_distances(user_service_matrix.T, metric='manhattan')

manhattan_similarity = 1 / (1 + manhattan_dist_matrix)

manhattan_sim_df = pd.DataFrame(manhattan_similarity, index=user_service_matrix.columns, columns=user_service_matrix.columns)

In [73]:
def recommend_services_manhattan(user_id, user_service_matrix, similarity_matrix, top_n=3):
    if user_id not in user_service_matrix.index:
        return []

    subscribed_services = user_service_matrix.loc[user_id]
    subscribed_services = subscribed_services[subscribed_services > 0].index.tolist()

    service_similarities = similarity_matrix[subscribed_services].mean(axis=1)

    for service in subscribed_services:
        service_similarities.drop(service, inplace=True, errors='ignore')

    recommended_services = service_similarities.nlargest(top_n).index.tolist()

    return recommended_services

manhattan_sim_df.head()

service,10291,10292,459505104,459505105,459505106,459509901,459509902,459509903,9109,CLOUDPLAY,...,GMSPAZE,PLAYZONE,QUIZPRO,gemezz,goaly,halogame,linguisto,omantel,quizy,slypee
service,,,,,,,,,,,,,,,,,,,,,
10291,1.000000,0.000100,0.000115,0.000118,0.000117,0.000119,0.000116,0.000119,0.000124,0.000099,...,0.000124,0.000117,0.000115,0.000048,0.000077,0.000120,0.000114,0.000120,0.000110,0.000063
10292,0.000100,1.000000,0.000385,0.000423,0.000405,0.000436,0.000393,0.000430,0.000511,0.000249,...,0.000511,0.000407,0.000379,0.000068,0.000143,0.000438,0.000376,0.000448,0.000329,0.000101
459505104,0.000115,0.000385,1.000000,0.000950,0.000861,0.001015,0.000812,0.000983,0.001536,0.000369,...,0.001536,0.000870,0.000749,0.000075,0.000176,0.001022,0.000739,0.001080,0.000576,0.000117
459505105,0.000118,0.000423,0.000950,1.000000,0.001070,0.001318,0.000992,0.001267,0.002375,0.000403,...,0.002375,0.001088,0.000904,0.000076,0.000184,0.001337,0.000890,0.001437,0.000664,0.000120
459505106,0.000117,0.000405,0.000861,0.001070,1.000000,0.001161,0.000901,0.001115,0.001898,0.000387,...,0.001898,0.000976,0.000825,0.000076,0.000180,0.001171,0.000814,0.001247,0.000620,0.000118


In [74]:
sample_users_manhattan = very_low_df['msisdn'].sample(n=10, random_state=1)

manhattan_recommendations = {}
for user in sample_users_manhattan:
    recommended_services = recommend_services_manhattan(user, user_service_matrix, manhattan_sim_df)
    manhattan_recommendations[user] = recommended_services

manhattan_recommendations

{'66927283010': ['GALAYS', '9109', 'GEC'],
 '2078536652': ['GALAYS', '9109', 'GEC'],
 '96890961474': ['GALAYS', '9109', 'GEC'],
 '2078600815': ['GALAYS', '9109', 'GEC'],
 '66817595537': ['GALAYS', '9109', 'GEC'],
 '96891473528': ['GALAYS', '9109', 'GEC'],
 '639102048395': ['GALAYS', '9109', 'GEC'],
 '639129058204': ['GALAYS', '9109', 'GEC'],
 'SHDC-F87reHqX+06xd4kA2Hc50BRj0pr8ZHQwlhMXaMw2VvI=': ['GALAYS',
  '9109',
  'GEC'],
 'SHDC-MdV5t9dCZaPAk8Gom322tRjFY9uKfqajxDYpMF2eqow': ['GALAYS', '9109', 'GEC']}

In [75]:
manhattan_recommendations.values()

dict_values([['GALAYS', '9109', 'GEC'], ['GALAYS', '9109', 'GEC'], ['GALAYS', '9109', 'GEC'], ['GALAYS', '9109', 'GEC'], ['GALAYS', '9109', 'GEC'], ['GALAYS', '9109', 'GEC'], ['GALAYS', '9109', 'GEC'], ['GALAYS', '9109', 'GEC'], ['GALAYS', '9109', 'GEC'], ['GALAYS', '9109', 'GEC']])

In [76]:
very_low_manhattan = very_low_df.copy()

very_low_manhattan['Recommended_Services'] = very_low_manhattan['msisdn'].apply(
    lambda x: recommend_services_manhattan(x, user_service_matrix, manhattan_sim_df))

very_low_manhattan[['msisdn', 'service', 'Recommended_Services']].head()

,msisdn,service,Recommended_Services
0,2020008781,slypee,"[GALAYS, 9109, GEC]"
1,2020010903,slypee,"[GALAYS, 9109, GEC]"
2,2020023548,slypee,"[GALAYS, 9109, GEC]"
3,2020025070,slypee,"[GALAYS, 9109, GEC]"
4,2020031236,slypee,"[GALAYS, 9109, GEC]"


## Low

In [77]:
low_expanded = low_df.assign(service=low_df['service'].str.split(', ')).explode('service')

user_service_matrix = pd.pivot_table(low_expanded, index='msisdn', columns='service', aggfunc='size', fill_value=0)

user_service_matrix.shape, user_service_matrix.head()

((1059, 13),
 service     10291  10292  CLOUDPLAY  GAMESPOT  PLAYZONE  QUIZPRO  gemezz  \
 msisdn                                                                     
 2022000108      0      0          0         0         0        0       0   
 2022007595      0      0          0         0         0        0       0   
 2022105544      0      0          0         0         0        0       0   
 2022261544      0      0          0         0         0        0       0   
 2022355232      0      0          0         0         0        0       0   
 
 service     goaly  halogame  linguisto  omantel  quizy  slypee  
 msisdn                                                          
 2022000108      0         0          0        0      0       1  
 2022007595      0         0          0        0      0       1  
 2022105544      0         0          0        0      0       1  
 2022261544      0         0          0        0      0       1  
 2022355232      0         0          0        0  

In [78]:
cosine_sim = cosine_similarity(user_service_matrix.T)

cosine_sim_df = pd.DataFrame(cosine_sim, index=user_service_matrix.columns, columns=user_service_matrix.columns)

cosine_sim_df.iloc[:10, :10]

service,10291,10292,CLOUDPLAY,GAMESPOT,PLAYZONE,QUIZPRO,gemezz,goaly,halogame,linguisto
service,,,,,,,,,,
10291,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
10292,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
CLOUDPLAY,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
GAMESPOT,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.000000
PLAYZONE,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.000000
QUIZPRO,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.000000
gemezz,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.000000
goaly,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.034353
halogame,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.000000


In [79]:
sample_users = low_df['msisdn'].sample(n=10, random_state=1)

recommendations = {}
for user in sample_users:
    recommended_services = recommend_services_cosine(user, user_service_matrix, cosine_sim_df, 2)
    recommendations[user] = recommended_services

recommendations

{'96891201488': ['linguisto', '10291'],
 '96896110051': ['10291', '10292'],
 '96898593383': ['linguisto', '10291'],
 '96898937989': ['linguisto', '10291'],
 '96893999041': ['omantel', 'goaly'],
 '96892249022': ['linguisto', '10291'],
 '96893158888': ['linguisto', '10291'],
 '96898842181': ['linguisto', '10291'],
 '96893257363': ['linguisto', '10291'],
 '96872111347': ['linguisto', '10291']}

In [80]:
low_df['Recommended_Services'] = low_df['msisdn'].apply(lambda x: recommend_services_cosine(x, user_service_matrix, cosine_sim_df))

low_df[['msisdn', 'service', 'Recommended_Services']].head()

,msisdn,service,Recommended_Services
357,2022000108,slypee,"[10291, 10292]"
362,2022007595,slypee,"[10291, 10292]"
386,2022105544,slypee,"[10291, 10292]"
571,2022261544,slypee,"[10291, 10292]"
670,2022355232,slypee,"[10291, 10292]"


In [81]:
low_df['Recommended_Services'].value_counts()

Recommended_Services
[linguisto, 10291]    667
[omantel, goaly]      194
[10291, 10292]        145
[goaly, 10291]         27
[omantel, 10291]       13
[10292, CLOUDPLAY]     10
[10291, CLOUDPLAY]      3
Name: count, dtype: int64

In [82]:
low_df[low_df['service'].str.contains(',')]

,msisdn,subscription_period_days,revenue_rupiah,country,operator,service,source,status,cycle,adnet,...,unsubs_date,service_price,currency,publisher,attempt_charging,success_billing,CLV,revenue_usd,Segment,Recommended_Services
30280,96871900330,225.0,5.757706e+05,OM,"omantel, linguisto","linguisto, omantel",cp1,1,daily,forest,...,2023-12-12,300,Baisa,lumos,50,48,2558.980288,36.849316,Low,"[goaly, 10291]"
30518,96872258102,142.0,1.559379e+05,OM,omantel,"linguisto, goaly",cp1,1,daily,"forest, fmd",...,2023-12-12,300,Baisa,lumos,15,13,1098.153953,9.980023,Low,"[omantel, 10291]"
30677,96872513704,80.0,1.559379e+05,OM,"linguisto, omantel","omantel, linguisto",cp1,1,daily,forest,...,2023-12-12,300,Baisa,lumos,13,13,1949.223266,9.980023,Low,"[goaly, 10291]"
30942,96872767787,45.0,4.798088e+04,OM,omantel,"linguisto, goaly",cp1,1,daily,"forest, fmd",...,2023-12-12,300,Baisa,lumos,4,4,1066.241787,3.070776,Low,"[omantel, 10291]"
31273,96890158279,63.0,1.079570e+05,OM,omantel,"linguisto, goaly",cp1,1,daily,"forest, adcuesta",...,2023-12-12,300,Baisa,lumos,11,9,1713.602871,6.909247,Low,"[omantel, 10291]"
31313,96890191109,277.0,1.079570e+06,OM,"omantel, linguisto","linguisto, omantel",cp1,1,daily,forest,...,2023-12-12,300,Baisa,lumos,92,90,3897.363931,69.092468,Low,"[goaly, 10291]"
31360,96890606730,57.0,3.838470e+05,OM,omantel,"goaly, linguisto",cp1,-1,daily,"fmd, forest",...,2023-12-12,300,Baisa,lumos,32,32,6734.158653,24.566211,Low,"[omantel, 10291]"
31737,96890979408,114.0,1.199522e+05,OM,"linguisto, omantel","omantel, linguisto",cp1,1,daily,forest,...,2023-12-12,300,Baisa,lumos,12,10,1052.212289,7.676941,Low,"[goaly, 10291]"
31855,96891125551,84.0,1.439426e+05,OM,"omantel, linguisto","linguisto, omantel",cp1,-1,daily,forest,...,2023-12-12,300,Baisa,lumos,14,12,1713.602871,9.212329,Low,"[goaly, 10291]"
32464,96891437945,243.0,5.517801e+05,OM,"linguisto, omantel","omantel, linguisto",cp1,1,daily,forest,...,2023-12-12,300,Baisa,lumos,48,46,2270.700101,35.313928,Low,"[goaly, 10291]"


## Medium

In [83]:
medium_expanded = medium_df.assign(service=medium_df['service'].str.split(', ')).explode('service')

user_service_matrix = pd.pivot_table(medium_expanded, index='msisdn', columns='service', aggfunc='size', fill_value=0)

user_service_matrix.shape, user_service_matrix.head()

((1104, 4),
 service      gemezz  goaly  linguisto  omantel
 msisdn                                        
 96871100702       0      1          0        0
 96871102317       0      1          0        0
 96871112961       0      1          0        0
 96871116514       0      1          0        0
 96871140839       0      1          0        0)

In [84]:
cosine_sim = cosine_similarity(user_service_matrix.T)

cosine_sim_df = pd.DataFrame(cosine_sim, index=user_service_matrix.columns, columns=user_service_matrix.columns)

cosine_sim_df.iloc[:10, :10]

service,gemezz,goaly,linguisto,omantel
service,,,,
gemezz,1.0,0.000000,0.000000,0.000000
goaly,0.0,1.000000,0.005688,0.010829
linguisto,0.0,0.005688,1.000000,0.459573
omantel,0.0,0.010829,0.459573,1.000000


In [85]:
sample_users = medium_df['msisdn'].sample(n=10, random_state=1)

recommendations = {}
for user in sample_users:
    recommended_services = recommend_services_cosine(user, user_service_matrix, cosine_sim_df)
    recommendations[user] = recommended_services

recommendations

{'96898090285': ['omantel', 'linguisto'],
 '96899578132': ['omantel', 'linguisto'],
 '96890628001': ['omantel', 'linguisto'],
 '96898090973': ['omantel', 'linguisto'],
 '96898916131': ['omantel', 'linguisto'],
 '96892332682': ['omantel', 'linguisto'],
 '96898558626': ['omantel', 'linguisto'],
 '96890655360': ['omantel', 'linguisto'],
 '96891917562': ['omantel', 'linguisto'],
 '96892660695': ['omantel', 'linguisto']}

In [86]:
medium_df['Recommended_Services'] = medium_df['msisdn'].apply(lambda x: recommend_services_cosine(x, user_service_matrix, cosine_sim_df))
medium_df[['msisdn', 'service', 'Recommended_Services']].head()

,msisdn,service,Recommended_Services
29917,96871100702,goaly,"[omantel, linguisto]"
29919,96871102317,goaly,"[omantel, linguisto]"
29927,96871112961,goaly,"[omantel, linguisto]"
29933,96871116514,goaly,"[omantel, linguisto]"
29963,96871140839,goaly,"[omantel, linguisto]"


In [87]:
medium_df['Recommended_Services'].value_counts()

Recommended_Services
[omantel, linguisto]    1064
[omantel, goaly]          21
[goaly, linguisto]        10
[goaly, gemezz]            7
[omantel, gemezz]          1
[linguisto, gemezz]        1
Name: count, dtype: int64

In [88]:
medium_df[medium_df['service'].str.contains(',')]

,msisdn,subscription_period_days,revenue_rupiah,country,operator,service,source,status,cycle,adnet,...,unsubs_date,service_price,currency,publisher,attempt_charging,success_billing,CLV,revenue_usd,Segment,Recommended_Services
31190,96890112315,327.0,1.199522e+07,OM,omantel,"goaly, linguisto",cp1,-1,daily,"kee, forest",...,2023-12-12,300,Rial,lumos,3,1,36682.630275,767.694086,Medium,"[omantel, gemezz]"
34431,96892724258,175.0,1.967216e+06,OM,"linguisto, omantel","omantel, linguisto",cp1,1,daily,forest,...,2023-12-12,300,Baisa,lumos,166,164,11241.234837,125.901830,Medium,"[goaly, gemezz]"
34478,96892750798,278.0,5.625758e+06,OM,"omantel, linguisto","linguisto, omantel",cp1,1,daily,forest,...,2023-12-12,300,Baisa,lumos,469,469,20236.540385,360.048527,Medium,"[goaly, gemezz]"
34495,96892755650,150.0,1.859259e+06,OM,"linguisto, omantel","omantel, linguisto",cp1,1,daily,forest,...,2023-12-12,300,Baisa,lumos,155,155,12395.060770,118.992583,Medium,"[goaly, gemezz]"
35290,96893354620,305.0,2.399044e+07,OM,"omantel, linguisto","goaly, omantel",cp1,1,daily,"fmd, forest",...,2023-12-12,300,Rial,lumos,4,2,78657.180984,1535.388173,Medium,"[linguisto, gemezz]"
37415,96898269669,234.0,4.438231e+06,OM,"linguisto, omantel","omantel, linguisto",cp1,1,daily,forest,...,2023-12-12,300,Baisa,lumos,370,370,18966.801013,284.046812,Medium,"[goaly, gemezz]"
37935,96898914466,86.0,1.043584e+06,OM,"omantel, linguisto","linguisto, omantel",cp1,1,daily,forest,...,2023-12-12,300,Baisa,lumos,87,87,12134.699403,66.789386,Medium,"[goaly, gemezz]"
38796,96899341023,77.0,8.276702e+05,OM,"omantel, linguisto","linguisto, omantel",cp1,1,daily,forest,...,2023-12-12,300,Baisa,lumos,69,69,10748.963466,52.970892,Medium,"[goaly, gemezz]"
38819,96899359827,144.0,2.111159e+06,OM,"omantel, linguisto","linguisto, omantel",cp1,-1,daily,forest,...,2023-12-12,300,Baisa,lumos,176,176,14660.824567,135.114159,Medium,"[goaly, gemezz]"


## High

In [89]:
high_expanded = high_df.assign(service=high_df['service'].str.split(', ')).explode('service')
user_service_matrix = pd.pivot_table(high_expanded, index='msisdn', columns='service', aggfunc='size', fill_value=0)
user_service_matrix.shape, user_service_matrix.head()

((1180, 3),
 service      goaly  linguisto  omantel
 msisdn                                
 96871040071      1          0        0
 96871110484      1          0        0
 96871116088      1          0        0
 96871116607      1          0        0
 96871126990      1          0        0)

In [90]:
cosine_sim = cosine_similarity(user_service_matrix.T)
cosine_sim_df = pd.DataFrame(cosine_sim, index=user_service_matrix.columns, columns=user_service_matrix.columns)
cosine_sim_df.iloc[:10, :10]

service,goaly,linguisto,omantel
service,,,
goaly,1.000000,0.041169,0.029111
linguisto,0.041169,1.000000,0.000000
omantel,0.029111,0.000000,1.000000


In [91]:
sample_users = high_df['msisdn'].sample(n=10, random_state=1)

recommendations = {}
for user in sample_users:
    recommended_services = recommend_services_cosine(user, user_service_matrix, cosine_sim_df)
    recommendations[user] = recommended_services

recommendations

{'96893343842': ['linguisto', 'omantel'],
 '96892747350': ['linguisto', 'omantel'],
 '96899506336': ['linguisto', 'omantel'],
 '96891925634': ['linguisto', 'omantel'],
 '96899477793': ['linguisto', 'omantel'],
 '96890604803': ['linguisto', 'omantel'],
 '96893500574': ['linguisto', 'omantel'],
 '96899710390': ['linguisto', 'omantel'],
 '96898538907': ['linguisto', 'omantel'],
 '96892744516': ['linguisto', 'omantel']}

In [92]:
high_df['Recommended_Services'] = high_df['msisdn'].apply(lambda x: recommend_services_cosine(x, user_service_matrix, cosine_sim_df))
high_df[['msisdn', 'service', 'Recommended_Services']].head()

,msisdn,service,Recommended_Services
29916,96871040071,goaly,"[linguisto, omantel]"
29924,96871110484,goaly,"[linguisto, omantel]"
29929,96871116088,goaly,"[linguisto, omantel]"
29935,96871116607,goaly,"[linguisto, omantel]"
29946,96871126990,goaly,"[linguisto, omantel]"


In [93]:
high_df['Recommended_Services'].value_counts()

Recommended_Services
[linguisto, omantel]    1177
[omantel]                  2
[linguisto]                1
Name: count, dtype: int64